<a href="https://colab.research.google.com/github/Iz-castro/analise_musical/blob/main/Fuga_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from music21 import converter, stream, note, interval, meter

#**Algoritmos do Gilberto**

In [2]:
# Função para calcular intervalos entre as notas de uma sequência
def calcular_intervalos(notas):
    intervalos = []
    for i in range(len(notas) - 1):
        intvl = interval.Interval(noteStart=notas[i], noteEnd=notas[i+1])
        intervalos.append(intvl.semitones)
    return intervalos

In [3]:
# Função para realizar a análise GPR (Grupo)
def gpr_analise(notas):
    # Exemplo: Agrupando notas com base em intervalos maiores que 2 semitons
    grupos = []
    grupo_atual = [notas[0]]
    for i in range(1, len(notas)):
        intervalo_atual = interval.Interval(noteStart=notas[i-1], noteEnd=notas[i]).semitones
        if abs(intervalo_atual) > 2:  # Definir fronteiras de grupo
            grupos.append(grupo_atual)
            grupo_atual = []
        grupo_atual.append(notas[i])
    grupos.append(grupo_atual)  # Adicionar o último grupo
    return grupos

In [4]:
# Função para realizar a análise MS (Métrica)
def calcular_peso_metrico(evento):
    # Exemplo simples: peso maior para tempo forte
    if evento.beat == 1:
        return 3  # Tempo forte
    elif evento.beat == 3:
        return 2  # Meio do compasso
    else:
        return 1  # Tempo fraco

In [5]:
# Função para aplicar TSR (Redução Temporal)
def calcular_tsr(notas):
    pesos = [calcular_peso_metrico(n) for n in notas]
    return pesos

In [6]:
# Função para realizar a análise PRL (Redução Prolongacional)
def calcular_tensao_prolongacional(notas):
    # Exemplo simples: notas mais altas têm mais tensão
    tensoes = [evento.pitch.ps for evento in notas]
    return tensoes

In [7]:
# Função principal para realizar as análises e gerar o CSV
def analisar_partitura_completa(score):
    resultados_analise = []

    for i, part in enumerate(score.parts):
        voz = f"Voz {i + 1}"
        measures = part.getElementsByClass('Measure')

        for k, measure in enumerate(measures):
            notas_compasso = [n for n in measure.notes if isinstance(n, note.Note)]

            if notas_compasso:
                # Realizar as análises GPR, MS, TSR, PRL
                grupos = gpr_analise(notas_compasso)
                pesos_metricos = calcular_tsr(notas_compasso)
                tensao_prolongacional = calcular_tensao_prolongacional(notas_compasso)

                resultados_analise.append({
                    "voz": voz,
                    "compasso": k + 1,
                    "grupos": grupos,
                    "peso_metrico": pesos_metricos,
                    "tensao_prolongacional": tensao_prolongacional
                })

    return resultados_analise

In [8]:
# Carregar a partitura (exemplo)
score = converter.parse('Fugue1.musicxml')

In [9]:
# Aplicar a função de análise à partitura completa
resultados_analise = analisar_partitura_completa(score)

In [10]:
# Gerar o CSV com os resultados das análises GPR, MS, TSR e PRL
import csv
with open("analises_tese.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Voz", "Compasso", "Grupos (notas)", "Peso Métrico", "Tensão Prolongacional"])

    for res in resultados_analise:
        # Transformar os dados em strings para escrever no CSV
        grupos_str = '; '.join([', '.join([n.nameWithOctave for n in grupo]) for grupo in res["grupos"]])
        peso_metrico_str = ', '.join(map(str, res["peso_metrico"]))
        tensao_prolongacional_str = ', '.join(map(str, res["tensao_prolongacional"]))

        writer.writerow([res["voz"], res["compasso"], grupos_str, peso_metrico_str, tensao_prolongacional_str])

print("Análise gerada e exportada para 'analises_tese.csv'.")

Análise gerada e exportada para 'analises_tese.csv'.


#Pre Processando o CSV Original:

In [11]:
import csv

# Caminho do CSV original gerado
csv_original_path = "analises_tese.csv"

# Caminho do novo CSV pré-processado
csv_preprocessado_path = "analises_preprocessadas.csv"

In [12]:
# Função para converter notas do formato string para MIDI
def converter_notas_para_midi(notas_str):
    notas_lista = notas_str.split("; ")
    midi_values = []

    for notas_grupo in notas_lista:
        notas_individuais = notas_grupo.split(", ")  # Separar as notas dentro de cada grupo
        for nota_texto in notas_individuais:
            try:
                n = note.Note(nota_texto)
                midi_values.append(n.pitch.midi)  # Adicionar valor MIDI
            except Exception as e:
                print(f"Erro ao converter a nota {nota_texto}: {e}")
                midi_values.append(0)  # Tratar erros e adicionar pausa (valor 0)

    return midi_values

In [13]:
# Ler o CSV original e criar o pré-processado com as notas em formato MIDI
with open(csv_original_path, mode='r', newline='', encoding='utf-8') as file_in, \
        open(csv_preprocessado_path, mode='w', newline='', encoding='utf-8') as file_out:

    reader = csv.reader(file_in)
    writer = csv.writer(file_out)

    # Copiar o cabeçalho
    header = next(reader)
    writer.writerow(header)

    # Processar cada linha do CSV original
    for row in reader:
        # A coluna dos grupos de notas está no índice 2 (ajuste se necessário)
        grupos_notas_midi = converter_notas_para_midi(row[2])  # Converte as notas para MIDI
        grupos_notas_midi_str = ', '.join(map(str, grupos_notas_midi))  # Converte de volta para string para salvar no CSV

        # Substituir a coluna de notas pela versão em MIDI
        row[2] = grupos_notas_midi_str

        # Escrever no novo CSV
        writer.writerow(row)

print(f"CSV pré-processado gerado em: {csv_preprocessado_path}")

CSV pré-processado gerado em: analises_preprocessadas.csv


#**Dados Finais**

In [27]:
from music21 import converter, note, chord, stream, instrument, midi, meter
import csv

In [28]:
# Função para converter nomes de notas para números MIDI
def notas_para_midi(notas):
    return [note.Pitch(n).midi for n in notas]

# Função para calcular intervalos entre notas MIDI
def calcular_intervalos_midi(notas_midi):
    intervalos = []
    for i in range(len(notas_midi) - 1):
        intervalos.append(notas_midi[i+1] - notas_midi[i])
    return intervalos

In [33]:
# Função para detectar motivos e transposições e coletar todos os elementos
def detectar_elementos_completos(score):
    dados = []

    for i, part in enumerate(score.parts):
        voz = f"Voz {i + 1}"  # Nomear as vozes como "Voz 1", "Voz 2", etc.
        instr = part.getInstrument(returnDefault=False)
        nome_instrumento = instr.instrumentName if instr is not None else voz

        measures = part.getElementsByClass('Measure')

        for k, measure in enumerate(measures):
            for element in measure.notesAndRests:
                if isinstance(element, note.Note):
                    dados.append({
                        'pitch': element.pitch.midi,  # Access .midi through .pitch
                        'duration': element.duration.quarterLength,
                        'offset': element.offset,
                        'velocity': element.volume.velocity if element.volume.velocity is not None else 64,
                        'instrument': nome_instrumento,
                        'is_rest': False
                    })
                elif isinstance(element, note.Rest):
                    dados.append({
                        'pitch': None,
                        'duration': element.duration.quarterLength,
                        'offset': element.offset,
                        'velocity': 0,  # Velocidade nula para rests
                        'instrument': nome_instrumento,
                        'is_rest': True
                    })
                elif isinstance(element, chord.Chord):
                    dados.append({
                        'pitch': [n.pitch.midi for n in element.notes],  # Access .midi through .pitch for each note in chord
                        'duration': element.duration.quarterLength,
                        'offset': element.offset,
                        'velocity': element.volume.velocity if element.volume.velocity is not None else 64,
                        'instrument': nome_instrumento,
                        'is_rest': False
                    })
    return dados

In [34]:
# Carregar a partitura (exemplo)
score = converter.parse('Fugue1.musicxml')

In [35]:
# Detectar todos os elementos musicais
dados_completos = detectar_elementos_completos(score)

In [36]:
# Salvar os dados no CSV
csv_filename = 'dados_completos_musica.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["pitch", "duration", "offset", "velocity", "instrument", "is_rest"])
    writer.writeheader()

    for linha in dados_completos:
        writer.writerow(linha)

print(f"Dados completos exportados para '{csv_filename}'.")

Dados completos exportados para 'dados_completos_musica.csv'.


In [37]:
import pandas as pd

In [38]:
# Carregar o CSV gerado anteriormente
csv_filename = 'dados_completos_musica.csv'
df = pd.read_csv(csv_filename)

In [39]:
# Mapeamento para a coluna "duration"
def ajustar_duration(duracao):
    if duracao == 4:
        return 1
    elif duracao == 2:
        return 2
    elif duracao == 1:
        return 4
    elif duracao == 0.5:
        return 8
    elif duracao == 0.75:
        return 12
    elif duracao == 0.25:
        return 16
    elif duracao == 0.125:
        return 32
    elif duracao == 0.0625:
        return 64
    elif duracao == 0.03125:
        return 128
    else:
        return duracao  # Manter o valor original se não estiver no mapeamento

In [40]:
# Aplicar o ajuste na coluna "duration"
df['duration'] = df['duration'].apply(ajustar_duration)

# Converter a coluna "is_rest" para 0 (não é um descanso) e 1 (é um descanso)
df['is_rest'] = df['is_rest'].apply(lambda x: 1 if x else 0)

# Manter o offset sem relação direta ao metrônomo, apenas convertendo em milissegundos
df['offset'] = df['offset'] * 1000  # Assumindo que o offset original é em segundos, multiplicamos por 1000 para milissegundos

In [41]:
# Salvar o CSV pré-processado
csv_filename_processado = 'dados_preprocessados_completos_musica.csv'
df.to_csv(csv_filename_processado, index=False)

print(f"CSV pré-processado salvo como '{csv_filename_processado}'.")

CSV pré-processado salvo como 'dados_preprocessados_completos_musica.csv'.
